# Bài 1: Cài đặt giải thuật Apriori và ứng dụng giải thuật này cho 1 bộ dữ liệu

In [43]:
def create_candidates(data):
    """ Hàm này tạo các mục phổ biến kích thước 1 từ tập dữ liệu."""
    candidates = set()
    for transaction in data:
        for item in transaction:
            candidates.add(frozenset([item]))
    return candidates

def prune_candidates(candidates, data, min_support):
    """Loại bỏ các mục không phổ biến từ tập ứng viên dựa trên mức hỗ trợ tối thiểu."""
    candidate_counts = {}
    for transaction in data:
        for candidate in candidates:
            if candidate.issubset(transaction):
                candidate_counts[candidate] = candidate_counts.get(candidate, 0) + 1

    num_transactions = len(data)
    frequent_candidates = []
    support_data = {}
    for candidate, count in candidate_counts.items():
        support = count / num_transactions
        if support >= min_support:
            frequent_candidates.append(candidate)
        support_data[candidate] = support

    return frequent_candidates, support_data

def create_new_candidates(old_candidates, k):
    """ Tạo các tập ứng viên mới từ các tập phổ biến cũ"""
    new_candidates = []
    num_old_candidates = len(old_candidates)
    for i in range(num_old_candidates):
        for j in range(i + 1, num_old_candidates):
            itemset1 = list(old_candidates[i])[:k - 2]
            itemset2 = list(old_candidates[j])[:k - 2]
            itemset1.sort()
            itemset2.sort()
            if itemset1 == itemset2:
                new_candidates.append(old_candidates[i] | old_candidates[j])
    return new_candidates

def apriori(data, min_support):
    candidates = create_candidates(data)
    frequent_items, support_data = prune_candidates(candidates, data, min_support)
    all_frequent_items = [frequent_items]
    k = 2
    while len(all_frequent_items[-1]) > 0:
        new_candidates = create_new_candidates(all_frequent_items[-1], k)
        frequent_items, new_support_data = prune_candidates(new_candidates, data, min_support)
        support_data.update(new_support_data)
        all_frequent_items.append(frequent_items)
        k += 1
    return all_frequent_items, support_data

def generate_rules(frequent_itemsets, support_data, min_confidence):
    """ Tạo các luật từ các tập phổ biến và dữ liệu hỗ trợ."""
    rules = []
    for k_itemset in frequent_itemsets[1:]:
        for itemset in k_itemset:
            for item in itemset:
                antecedent = itemset - set([item])
                consequent = set([item])
                confidence = support_data[itemset] / support_data[antecedent]
                if confidence >= min_confidence:
                    rules.append((antecedent, consequent, confidence))
    return rules




In [44]:
data = [['B', 'C', 'D'],
        ['A', 'C'],
        ['A', 'D'],
        ['B', 'D']]

In [45]:
min_support = 0.2

frequent_itemsets, support_data = apriori(data, min_support)

print("Các tập mặt hàng phổ biến:")
for k, itemsets in enumerate(frequent_itemsets):
    print(f"Kích thước {k+1}: {itemsets}")

print("\nDữ liệu hỗ trợ:")
for itemset, support in support_data.items():
    print(f"{itemset}: {support}")
    
min_confidence = 0.8
rules = generate_rules(frequent_itemsets, support_data, min_confidence)

print("Các luật kết hợp:")
for antecedent, consequent, confidence in rules:
    print(f"{list(antecedent)} => {list(consequent)} (Confidence: {confidence})")

Các tập mặt hàng phổ biến:
Kích thước 1: [frozenset({'B'}), frozenset({'D'}), frozenset({'C'}), frozenset({'A'})]
Kích thước 2: [frozenset({'B', 'D'}), frozenset({'B', 'C'}), frozenset({'D', 'C'}), frozenset({'A', 'C'}), frozenset({'D', 'A'})]
Kích thước 3: [frozenset({'B', 'D', 'C'})]
Kích thước 4: []

Dữ liệu hỗ trợ:
frozenset({'B'}): 0.5
frozenset({'D'}): 0.75
frozenset({'C'}): 0.5
frozenset({'A'}): 0.5
frozenset({'B', 'D'}): 0.5
frozenset({'B', 'C'}): 0.25
frozenset({'D', 'C'}): 0.25
frozenset({'A', 'C'}): 0.25
frozenset({'D', 'A'}): 0.25
frozenset({'B', 'D', 'C'}): 0.25
Các luật kết hợp:
['B'] => ['D'] (Confidence: 1.0)
['D', 'C'] => ['B'] (Confidence: 1.0)
['B', 'C'] => ['D'] (Confidence: 1.0)


### Kiểm tra lại với hàm thư viện

In [46]:
# !pip install mlxtend

In [47]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd


encoder = TransactionEncoder()
transactions_encoded = encoder.fit(data).transform(data)
df = pd.DataFrame(transactions_encoded, columns=encoder.columns_)
frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)
print(frequent_itemsets)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
for i in rules.index:
    if rules.loc[i, "confidence"]>=min_confidence:
        print(rules.loc[i, 'antecedents'], '=>', rules.loc[i, 'consequents'], f'(Confidence: {rules.loc[i, "confidence"]})')

   support   itemsets
0     0.50        (A)
1     0.50        (B)
2     0.50        (C)
3     0.75        (D)
4     0.25     (C, A)
5     0.25     (D, A)
6     0.25     (B, C)
7     0.50     (B, D)
8     0.25     (D, C)
9     0.25  (B, D, C)
frozenset({'B'}) => frozenset({'D'}) (Confidence: 1.0)
frozenset({'B', 'C'}) => frozenset({'D'}) (Confidence: 1.0)
frozenset({'D', 'C'}) => frozenset({'B'}) (Confidence: 1.0)


# Bài 2:  Cài đặt giải thuật FP-Growth và ứng dụng giải thuật này cho 1 bộ dữ liệu

In [48]:
class TreeNode:
    def __init__(self, item, frequency, parent):
        self.item = item
        self.frequency = frequency
        self.parent = parent
        self.children = {}
        self.next_link = None

def construct_tree(dataset, min_support):
    """ hàm này xây dựng cây FP-Growth từ tập dữ liệu với min_support"""
    header_table = {}
    for transaction in dataset:
        for item in transaction:
            header_table[item] = header_table.get(item, 0) + dataset[transaction]

    for item in list(header_table.keys()):
        if header_table[item] < min_support:
            del(header_table[item])

    frequent_items = set(header_table.keys())

    if len(frequent_items) == 0:
        return None, None

    for item in header_table:
        header_table[item] = [header_table[item], None]

    fp_tree = TreeNode("Null", 1, None)
    for transaction, frequency in dataset.items():
        transaction_sorted = []
        for item in transaction:
            if item in frequent_items:
                transaction_sorted.append(item)
        if len(transaction_sorted) > 0:
            update_tree(fp_tree, transaction_sorted, header_table, frequency)

    return fp_tree, header_table

def update_tree(current_node, transaction_sorted, header_table, frequency):
    """hàm này cập nhật cây FP-Growth với một giao dịch đã được sắp xếp và một bảng tiêu đề."""
    if transaction_sorted[0] in current_node.children:
        current_node.children[transaction_sorted[0]].frequency += frequency
    else:
        current_node.children[transaction_sorted[0]] = TreeNode(transaction_sorted[0], frequency, current_node)

        if header_table[transaction_sorted[0]][1] is None:
            header_table[transaction_sorted[0]][1] = current_node.children[transaction_sorted[0]]
        else:
            update_header(header_table[transaction_sorted[0]][1], current_node.children[transaction_sorted[0]])

    if len(transaction_sorted) > 1:
        update_tree(current_node.children[transaction_sorted[0]], transaction_sorted[1:], header_table, frequency)

def update_header(node_to_test, target_node):
    """hàm này cập nhật liên kết tiếp theo của một nút trong cây FP-Growth."""
    while node_to_test.next_link is not None:
        node_to_test = node_to_test.next_link
    node_to_test.next_link = target_node

def ascend_tree(node, prefix_path):
    """ hàm này tạo một đường dẫn từ một nút trong cây FP-Growth đến gốc."""
    if node.parent is not None:
        prefix_path.append(node.item)
        ascend_tree(node.parent, prefix_path)

def find_prefix_path(base_item, header_table):
    """Hàm này tìm các đường dẫn tiền tố của một mục trong cây FP-Growth và bảng tiêu đề."""
    tree_node = header_table[base_item][1]
    prefix_paths = {}
    while tree_node is not None:
        prefix_path = []
        ascend_tree(tree_node, prefix_path)
        if len(prefix_path) > 1:
            prefix_paths[frozenset(prefix_path[1:])] = tree_node.frequency
        tree_node = tree_node.next_link
    return prefix_paths

def mine_patterns(header_table, min_support, prefix, frequent_itemsets):
    """ hàm này tìm các tập phổ biến từ cây FP-Growth và bảng tiêu đề."""
    bigL = [v[0] for v in sorted(header_table.items(), key=lambda p: p[1][0])]

    for base_item in bigL:
        new_frequent_set = prefix.copy()
        new_frequent_set.add(base_item)
        frequent_itemsets.append(new_frequent_set)
        conditional_tree_paths = find_prefix_path(base_item, header_table)
        conditional_tree, conditional_header = construct_tree(conditional_tree_paths, min_support)
        if conditional_header is not None:
            mine_patterns(conditional_header, min_support, new_frequent_set, frequent_itemsets)

def fpgrowth(dataset, min_support):
    fp_tree, header_table = construct_tree(dataset, min_support)
    frequent_itemsets = []
    mine_patterns(header_table, min_support, set([]), frequent_itemsets)
    return frequent_itemsets

def generate_rules(frequent_itemsets, min_confidence):
    rules = []
    for itemset in frequent_itemsets:
        if len(itemset) > 1:
            rules.extend(generate_rules_from_itemset(itemset, min_confidence))
    return rules

def generate_rules_from_itemset(itemset, min_confidence):
    rules = []
    for item in itemset:
        antecedent = frozenset([item])
        consequent = itemset - antecedent
        confidence = support_data[frozenset(itemset)] / support_data[antecedent]
        if confidence >= min_confidence:
            rules.append((antecedent, consequent, confidence))
    return rules


transaction_counts = {}
for transaction in data:
    transaction_counts[frozenset(transaction)] = transaction_counts.get(frozenset(transaction), 0) + 1

min_support = 2 
frequent_itemsets = fpgrowth(transaction_counts, min_support)

print("Độ hỗ trợ của các tập mặt hàng phổ biến:")
for itemset in frequent_itemsets:
    support = support_data[frozenset(itemset)]
    if support >= min_support/len(data):
        print(f"{itemset}: {support}")

min_confidence = 0.7
rules = generate_rules(frequent_itemsets, min_confidence)

print("Các luật kết hợp:")
for antecedent, consequent, confidence in rules:
    print(f"{antecedent} => {consequent} (Confidence: {confidence})")

Độ hỗ trợ của các tập mặt hàng phổ biến:
{'B'}: 0.5
{'C'}: 0.5
{'A'}: 0.5
{'D'}: 0.75
{'B', 'D'}: 0.5
Các luật kết hợp:
frozenset({'B'}) => {'D'} (Confidence: 1.0)


### Kiểm tra lại

In [49]:
from mlxtend.frequent_patterns import fpgrowth


encoder = TransactionEncoder()
transactions_encoded = encoder.fit(data).transform(data)
df = pd.DataFrame(transactions_encoded, columns=encoder.columns_)
frequent_itemsets = fpgrowth(df, min_support=min_support/len(data), use_colnames=True)
print(frequent_itemsets)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
for i in rules.index:
    if rules.loc[i, "confidence"]>=min_confidence:
        print(rules.loc[i, 'antecedents'], '=>', rules.loc[i, 'consequents'], f'(Confidence: {rules.loc[i, "confidence"]})')

   support itemsets
0     0.75      (D)
1     0.50      (C)
2     0.50      (B)
3     0.50      (A)
4     0.50   (B, D)
frozenset({'B'}) => frozenset({'D'}) (Confidence: 1.0)


# Bài 3:  Cài đặt giải thuật cải tiến sau: (Source: A Sliding Window Based Approach for Mining Frequent Weighted Patterns Over Data Streams) 

In [50]:
class SWNNode:
    def __init__(self, name):
        self.name = name
        self.weight = 0
        self.children = {}
        self.pre = 0
        self.pos = 0

    def insert(self, transaction, tw):
        if transaction:
            first_item = transaction[0]
            remaining_items = transaction[1:]
            if first_item in self.children:
                self.children[first_item].weight += tw
            else:
                self.children[first_item] = SWNNode(first_item)
                self.children[first_item].weight = tw
            self.children[first_item].insert(remaining_items, tw)

In [51]:
class SWNTree:
    def __init__(self):
        self.root = SWNNode('null')

    def insert_tree(self, transaction, tw):
        self.root.insert(transaction, tw)

    def generate_pre_pos(self, node=None, pre=0):
        if node is None:
            node = self.root
        node.pre = pre
        for child in node.children.values():
            pre += 1
            pre = self.generate_pre_pos(child, pre)
        node.pos = pre
        return pre

    def construct_swn_tree(self, db, tw):
        for transaction in db:
            transaction.sort(key=lambda item: -item[1])  # Sort by frequency
            items = [item[0] for item in transaction]  # Keep only the item names
            self.insert_tree(items, tw)
        self.generate_pre_pos()

In [52]:
db = [
    [('A', 2), ('B', 1), ('E', 1)],
    [('B', 3), ('c', 2)],
    [('A', 1), ('E', 2), ('F', 1)],
    [('B', 2), ('E', 1), ('c', 1)],
    [('A', 1), ('B', 1), ('c', 1), ('F', 2)]
]

tw = 1
swn_tree = SWNTree()
swn_tree.construct_swn_tree(db, tw)

# In ra cây SWN để kiểm tra
def print_tree(node, indent=0):
    print('  ' * indent + f'{node.name} (weight: {node.weight}, pre: {node.pre}, pos: {node.pos})')
    for child in node.children.values():
        print_tree(child, indent + 1)

print_tree(swn_tree.root)

null (weight: 0, pre: 0, pos: 14)
  A (weight: 1, pre: 1, pos: 3)
    B (weight: 1, pre: 2, pos: 3)
      E (weight: 1, pre: 3, pos: 3)
  B (weight: 2, pre: 4, pos: 7)
    c (weight: 1, pre: 5, pos: 5)
    E (weight: 1, pre: 6, pos: 7)
      c (weight: 1, pre: 7, pos: 7)
  E (weight: 1, pre: 8, pos: 10)
    A (weight: 1, pre: 9, pos: 10)
      F (weight: 1, pre: 10, pos: 10)
  F (weight: 1, pre: 11, pos: 14)
    A (weight: 1, pre: 12, pos: 14)
      B (weight: 1, pre: 13, pos: 14)
        c (weight: 1, pre: 14, pos: 14)


# END